In [1]:
import json
import requests
import os

In [17]:
def get_models() -> dict:
    url = "https://api.gpt.mws.ru/v1/models"
    api_key = os.getenv('API_KEY')
    headers = {
        "Authorization": f"Bearer {api_key}"  # Replace YOUR_API_KEY with your actual API key
    }
    response = requests.get(url, headers=headers)
    print(response.status_code)
    return response.json()

In [18]:
response = get_models()

200


In [19]:
models = [item['id'] for item in response['data']]
models

['qwen2.5-72b-instruct',
 'bge-m3',
 'llama-3.1-8b-instruct',
 'deepseek-r1-distill-qwen-32b',
 'llama-3.3-70b-instruct',
 'qwen2.5-coder-7b-instruct',
 'qwen2.5-32b-instruct',
 'gemma-3-27b-it',
 'mws-gpt-alpha']

In [ ]:
def call_model_complition(user_prompt: str, model_name: str, context: str, temperature: float = 0.2) -> dict:
    url = "https://api.gpt.mws.ru/v1/completions"
    api_key = os.getenv('API_KEY')
    headers = {
        "Authorization": f"Bearer {api_key}"  # Replace YOUR_API_KEY with your actual API key
    }
    prompt = f"""
Ты - ассистент по поиску информации в данных. Используй следующий контекст для ответа на вопрос пользователя.
Если ты считаешь, что контекста недостаточно для формирования ответа, ответь что не можешь помочь в решении данного вопоса
Контекст:
{context}

Запрос пользователя:
{user_prompt}

Ответ:
"""
    data = {
        "model": model_name,
        "prompt": prompt,
        "temperature": temperature,
        "max_tokens": 150,
        "top_p": 1,
        "frequency_penalty": 0,
        "presence_penalty": 0,
    }
    response = requests.post(url, headers=headers, json=data)
    if response.status_code != 200:
        print(model_name, response.status_code) 
    else: 
        return response.json()

In [ ]:
results = []
querys_path = 'querys.json'
with open(querys_path, 'r', encoding='utf') as f:
    prompts = json.load(f)
    
# Models testing
for model in models:
    if model == 'bge-m3':
        print('not doing bge')
        continue
    for prompt_dict in prompts:
        result = call_model_complition(prompt_dict['user_prompt'], model, prompt_dict['context'])
        if result is not None:
            results.append({
                "model": model,
                "user_prompt": prompt_dict['user_prompt'],
                "context": prompt_dict['context'],
                "response": result
            })
        else:
            results.append({
                "model": model,
                "user_prompt": prompt_dict['user_prompt'],
                "context": prompt_dict['context'],
                "error": "API call failed"
            })

with open('results.json', 'w', encoding='utf-8') as f:
    json.dump(results, f, ensure_ascii=False, indent=4)

not doing bge


In [19]:
url = "https://api.gpt.mws.ru/v1/embeddings"
API_key = os.getenv('API_KEY')
model = "bge-m3"
input = [
    "как у тебя дела",
    "Как жизнь",
    ]

def reqToEmbading(url, API_key, model, input):
    headers = {
        "Authorization": f"Bearer {API_key}"
    }
    data = {
        "model": model,
        "input": input
    }

    response = requests.post(url, headers=headers, json=data)
    return response.json()
  
response = reqToEmbading(url, API_key, model, input)

In [ ]:
model_name = 'qwen2.5-72b-instruct'
url = "https://api.gpt.mws.ru/v1/chat/completions"
api_key = os.getenv("API_KEY")
headers = {
    "Authorization": f"Bearer {api_key}"
}
context = """Как проверить последние пополнения и списания
1. Откройте Расходы


2. По умолчанию отобразятся все операции: и списания, и пополнения


3. Нажмите на вкладку Списания или вкладку Пополнения, чтобы посмотреть только один тип операций

Как заказать детализацию
В детализации есть полные данные о том, как вы пользовались услугами связи в выбранный период. В ней вы найдёте все даты и суммы списаний и пополнений, список всех звонков, интернет-сессий и СМС.

Все события в детализации идут по порядку, начиная с самых ранних. Если вы активно пользуетесь интернетом, у вас будет много записей о выходах в сеть. В этом случае данные о звонках могут находиться между ними.

Как скачать детализацию или получить её на email:

1. Откройте Расходы


2. Нажмите ↓ Детализация


3. Выберите Отчёт по расходам


4. Выберите период и формат документа: PDF или XLSX


5. Если хотите сразу загрузить детализацию, нажмите Скачать. Чтобы получить документ на электронную почту, введите свой адрес и нажмите Отправить на email

Детализацию давностью от полугода до трёх лет можно получить только на email


Ещё один способ заказать детализацию ― в Личном кабинете. Инструкция есть в статье Как посмотреть детализацию расходов

Как оставаться на связи при минусе
Есть несколько способов пользоваться номером при отрицательном балансе. Вы можете посмотреть доступные опции и подключить их в разделе Каталог > Услуги > Возможности при нуле

Как проверить баланс и пакеты по тарифу
На главном экране, который открывается при входе в приложение Мой МТС, вы видите все самые важные цифры: 

сумму на балансе
оставшиеся минуты 
гигабайты интернета
СМС
Под остатком каждого пакета на счётчике написаны дни, оставшиеся до его обновления. Если вы израсходовали весь пакет или у вас безлимит, дни до обновления не показываются


Если пакеты по тарифу заканчиваются, о том, как их обновить, рассказываем в статье"""
system_prompt = f"""Ты ассистент для суммаризации информации
Твоя задача - суммаризировать текст, включив в него все необходимые ключевые данные
Не отвечай на вопросы в тексте и не добавляй новую информацию
Суммаризация не должна превышать 100 токенов"""
user_prompt = f"Текст: {context[:300]}"
data = {
        "model": model_name,
        "messages" : [
            {"role" : "system", "content" : system_prompt},
            {"role" : "use", "content": user_prompt}
        ],
        "temperature": 0,
        "max_tokens": 100,
        "n" : 1
    }
response = requests.post(url, headers=headers, json=data)
response

<Response [200]>

In [40]:
print(context[:300])

Как проверить последние пополнения и списания
1. Откройте Расходы


2. По умолчанию отобразятся все операции: и списания, и пополнения


3. Нажмите на вкладку Списания или вкладку Пополнения, чтобы посмотреть только один тип операций

Как заказать детализацию
В детализации есть полные данные о том, 


In [38]:
result = response.json()
result

{'id': 'chatcmpl-3068cf53f44e4b16ae13e272ead70c27',
 'created': 1745072647,
 'model': 'qwen2.5-72b-instruct',
 'object': 'chat.completion',
 'system_fingerprint': None,
 'choices': [{'finish_reason': 'stop',
   'index': 0,
   'message': {'content': 'Для проверки операций открой раздел "Расходы", где по умолчанию видны все транзакции. Выберите вкладку "Списания" или "Пополнения" для конкретного типа операций. Заказ детализации предоставит полные данные о транзакциях.',
    'role': 'assistant',
    'tool_calls': None,
    'function_call': None}}],
 'usage': {'completion_tokens': 77,
  'prompt_tokens': 184,
  'total_tokens': 261,
  'completion_tokens_details': None,
  'prompt_tokens_details': None},
 'service_tier': None,
 'prompt_logprobs': None}

In [39]:
print(result['choices'][0]['message']['content'])

Для проверки операций открой раздел "Расходы", где по умолчанию видны все транзакции. Выберите вкладку "Списания" или "Пополнения" для конкретного типа операций. Заказ детализации предоставит полные данные о транзакциях.


In [ ]:
model_name = 'deepseek-r1-distill-qwen-32b'
api_key = os.getenv('API_KEY')
url = "https://api.gpt.mws.ru/v1/chat/completions"
question = "Как проверить баланс и пакеты по тарифу"
context = ' Вы можете посмотреть доступные опции и подключить их в разделе Каталог > Услуги > Возможности при нуле Как проверить баланс и пакеты по тарифу\n\nТекст описывает, как проверить остаток интернет-пакета, дни до его обновления и условия тарифа на главном экране. Изменения условий оплаты или услуги могут произойти после бесплатного периода. Стоимость пакета списывается при подключении. Подробности тарифа, включая его название, размер пакетов и остатки, доступны\n\n Поэтому трафик по подписке может быть не виден сразу, но он точно есть Как это выглядит: в приложении Мой МТС в Личном кабинете В приложении и в Личном кабинете можно нажать на трафик, чтобы увидеть все подключённые пакеты\n\nразмер и остатки пакетов интернета, минут, СМС  Чтобы открыть полное описание своего тарифа:   1  На главной странице нажмите на его название   2  На открывшемся экране:\n\nВ разделе "Услуги" каталога можно подключить опции и проверить баланс. Нажмите на название тарифа для уточнения платы и даты списания. Кнопка "Настроить" доступна для тарифов с пакетами. Полное описание тарифа открывается при нажатии "О тарифе". Стоимость услуг и платные оп\n\n'
headers = {
            "Authorization": f"Bearer {api_key}"  # Replace YOUR_API_KEY with your actual API key
        }
system_prompt = f"""Ты ассистент для нахождения ответа на вопрос пользователя
Твоя задача - внимательно проанализировать запрос пользователя и найти информацию в предоставленном тексте
Постарайся отвечать кратко"""
user_prompt = f"""Информация: {context}
        
Запрос пользователя: {question}"""
        
data = {
    "model": 'deepseek-r1-distill-qwen-32b',
    "messages" : [
        {"role" : "system", "content" : system_prompt},
        {"role" : "user", "content": user_prompt}
    ],
    "temperature": 0.2,
    "n": 1,
    "frequency_penalty": 0,
    "presence_penalty": 0,
}

response = requests.post(url, headers=headers, json=data)
response

<Response [200]>

In [57]:
response.json()

{'id': 'chatcmpl-8c039cb4bfc342859d397f40e4990a5d',
 'created': 1745081211,
 'model': 'deepseek-r1-distill-qwen-32b',
 'object': 'chat.completion',
 'system_fingerprint': None,
 'choices': [{'finish_reason': 'stop',
   'index': 0,
   'message': {'content': '<think>\nХорошо, давайте разберемся, как помочь пользователю, который спрашивает: "Как проверить баланс и пакеты по тарифу". \n\nСначала я посмотрю на предоставленную информацию. В тексте говорится, что для проверки баланса и пакетов нужно использовать приложение "Мой МТС" или Личный кабинет. Там можно нажать на трафик, чтобы увидеть все подключенные пакеты, их размер и остатки. \n\nТакже упоминается, что в разделе "Услуги" каталога можно подключить опции и проверить баланс. Нажав на название тарифа, можно уточнить плату и дату списания. Кнопка "Настроить" доступна для тарифов с пакетами, а полное описание тарифа открывается при нажатии "О тарифе".\n\nПохоже, основные шаги такие:\n1. Зайти в приложение "Мой МТС" или Личный кабинет.\

In [58]:
print(response.json()['choices'][0]['message']['content'])

<think>
Хорошо, давайте разберемся, как помочь пользователю, который спрашивает: "Как проверить баланс и пакеты по тарифу". 

Сначала я посмотрю на предоставленную информацию. В тексте говорится, что для проверки баланса и пакетов нужно использовать приложение "Мой МТС" или Личный кабинет. Там можно нажать на трафик, чтобы увидеть все подключенные пакеты, их размер и остатки. 

Также упоминается, что в разделе "Услуги" каталога можно подключить опции и проверить баланс. Нажав на название тарифа, можно уточнить плату и дату списания. Кнопка "Настроить" доступна для тарифов с пакетами, а полное описание тарифа открывается при нажатии "О тарифе".

Похоже, основные шаги такие:
1. Зайти в приложение "Мой МТС" или Личный кабинет.
2. Нажать на трафик, чтобы увидеть пакеты.
3. Проверить раздел "Услуги" для дополнительной информации.

Может быть полезно упомянуть, что трафик по подписке не сразу виден, но он есть. Это важно, чтобы пользователь не волновался, если не сразу видит данные.

Теперь 